In [84]:
import cortex
import numpy as np

In [85]:
surfaces = dict( F = 'fMRI_story_F',
	G = 'fMRI_story_G',
	H = 'fMRI_story_H',
	I = 'fMRI_story_I',
	J = 'fMRI_story_J',
	K = 'fMRI_story_K',
	L = 'fMRI_story_L',
	M = 'fMRI_story_M',
	N = 'fMRI_story_N')

transforms = dict( F = 'F_ars_auto2',
	G = 'G_ars_auto2',
	H = 'H_ars_auto2',
	I = 'I_ars_auto2',
	J = 'J_ars_auto2',
	K = 'K_ars_auto2',
	L = 'L_ars_auto2',
	M = 'M_ars_auto2',
	N = 'N_ars_auto2')

vols = {}

In [86]:
def roi_to_allvoxels(rois, roi_array):
    num_folds = roi_array.shape[0]
    num_voxels = rois.item()['F']['all'].shape[0]
    allvoxel_array = np.zeros((num_folds, num_voxels))
    allvoxel_array[:] = np.nan
    roi_ids = np.where(rois.item()['F']['all'] == 1)
    for i in range(num_folds):
        allvoxel_array[i, roi_ids] = roi_array[i]
    return allvoxel_array

In [87]:
sub = 'F'
mask = cortex.db.get_mask(surfaces[sub], transforms[sub], 'thin')
rois = np.load('../HP_subj_roi_inds.npy', allow_pickle=True)

## Load Relevant Correlation and/or Accuracy Data

In [88]:
################################ 
#         linear data          #
################################
# lin_neighborhood_loaded = np.load('final_accs/acc_F_with_bert_layer_1_len_1_encoder_linear_accs_ALL.pkl', allow_pickle=True)
# lin_noneighborhood_loaded = np.load('final_accs/no_neighborhood_acc_F_with_bert_layer_1_len_1_encoder_linear_accs_ALL.pkl', allow_pickle=True)
# lin_output_loaded = np.load('encoder_preds/predict_F_with_bert_layer_1_len_1_encoder_linear.npy', allow_pickle=True).item()
# lin_preds_loaded = lin_output_loaded['preds_t']
# lin_corrs_loaded = lin_output_loaded['corrs_t']

################################ 
#       mlp_initial data       #
################################
# mlp_neighborhood_loaded = np.load('final_accs/acc_F_with_bert_layer_1_len_1_encoder_mlp_initial_accs_ALL.pkl', allow_pickle=True)
# mlp_noneighborhood_loaded = np.load('final_accs/no_neighborhood_acc_F_with_bert_layer_1_len_1_encoder_mlp_initial_accs_ALL.pkl', allow_pickle=True)
# mlp_output_loaded = np.load('encoder_preds/predict_F_with_bert_layer_1_len_1_encoder_mlp_initial.npy', allow_pickle=True).item()
# mlp_preds_loaded = mlp_output_loaded['preds_t']
# mlp_corrs_loaded = mlp_output_loaded['corrs_t']

################################ 
#       mlp_allvoxels data     #
################################
# mlp_allvoxels_neighborhood_loaded = np.load('final_accs/acc_F_with_bert_layer_1_len_1_encoder_mlp_allvoxels_accs_ALL.pkl', allow_pickle=True)
# mlp_allvoxels_noneighborhood_loaded = np.load('final_accs/no_neighborhood_acc_F_with_bert_layer_1_len_1_encoder_mlp_allvoxels_accs_ALL.pkl', allow_pickle=True)
# mlp_allvoxels_output_loaded = np.load('encoder_preds/predict_F_with_bert_layer_1_len_1_encoder_mlp_allvoxels.npy', allow_pickle=True).item()
# mlp_allvoxels_preds_loaded = mlp_allvoxels_output_loaded['preds_t']
# mlp_allvoxels_corrs_loaded = mlp_allvoxels_output_loaded['corrs_t']

If using only ROI correlations/accuracies, extend array size with nans for all other voxels:

In [89]:
# lin_noneighborhood_loaded = roi_to_allvoxels(rois, lin_noneighborhood_loaded)
# lin_corrs_loaded = roi_to_allvoxels(rois, lin_corrs_loaded)
# mlp_noneighborhood_loaded = roi_to_allvoxels(rois, mlp_noneighborhood_loaded)
# mlp_corrs_loaded = roi_to_allvoxels(rois, mlp_corrs_loaded)
# mlp_allvoxels_noneighborhood_loaded = roi_to_allvoxels(rois, mlp_allvoxels_noneighborhood_loaded)
# mlp_allvoxels_corrs_loaded = roi_to_allvoxels(rois, mlp_allvoxels_corrs_loaded)

In [90]:
### PLOTTING STD DEVIATIONS ####

In [91]:
def CV_ind(n, n_folds):
    ind = np.zeros((n))
    n_items = int(np.floor(n/n_folds))
    for i in range(0,n_folds -1):
        ind[i*n_items:(i+1)*n_items] = i
    ind[(n_folds-1)*n_items:] = (n_folds-1)
    return ind

def fold_voxels_stderr(ind_num, n_folds, preds):
    n = preds.shape[0]
    ind = CV_ind(n, n_folds)
    preds_ind = ind==ind_num
    fold_preds = preds[preds_ind]
    print(np.std(fold_preds, 0))
    where_stderr_zero = (np.std(fold_preds, 0) != 0)
    return where_stderr_zero

lin_fold0 = fold_voxels_stderr(0, 4, lin_preds_loaded)
lin_fold1 = fold_voxels_stderr(1, 4, lin_preds_loaded)
lin_fold2 = fold_voxels_stderr(2, 4, lin_preds_loaded)
lin_fold3 = fold_voxels_stderr(3, 4, lin_preds_loaded)

mlp_fold0 = fold_voxels_stderr(0, 4, mlp_preds_loaded)
mlp_fold1 = fold_voxels_stderr(1, 4, mlp_preds_loaded)
mlp_fold2 = fold_voxels_stderr(2, 4, mlp_preds_loaded)
mlp_fold3 = fold_voxels_stderr(3, 4, mlp_preds_loaded)

mlp_allvoxels_fold0 = fold_voxels_stderr(0, 4, mlp_allvoxels_preds_loaded)
mlp_allvoxels_fold1 = fold_voxels_stderr(1, 4, mlp_allvoxels_preds_loaded)
mlp_allvoxels_fold2 = fold_voxels_stderr(2, 4, mlp_allvoxels_preds_loaded)
mlp_allvoxels_fold3 = fold_voxels_stderr(3, 4, mlp_allvoxels_preds_loaded)

[0.01960691 0.01877794 0.02483443 ... 0.03295062 0.02268619 0.02836826]
[0.01693559 0.01628661 0.02249619 ... 0.01930797 0.02433178 0.02120906]
[0.15188601 0.15003005 0.13797689 ... 0.03861948 0.02386051 0.02168274]
[0.02583387 0.0260545  0.12440597 ... 0.01991488 0.025286   0.02357932]
[1.43834084e-11 0.00000000e+00 2.41319100e-11 ... 0.00000000e+00
 2.65030589e-11 2.25203539e-11]
[2.10542640e-11 1.70746850e-11 0.00000000e+00 ... 2.44932278e-11
 0.00000000e+00 1.93592797e-11]
[9.44232241e-12 2.08302569e-11 0.00000000e+00 ... 0.00000000e+00
 0.00000000e+00 9.44232241e-12]
[1.81894208e-11 0.00000000e+00 1.97424934e-11 ... 2.44370773e-11
 1.88386139e-11 9.39607805e-12]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0.00761665 0.01800956 0.02410695 ... 0.00380889 0.00443058 0.00304104]


In [92]:
# vols[sub+'_lin_fold0_stderr'] = cortex.Volume(lin_fold0, surfaces[sub], transforms[sub], mask=mask, vmin=0, vmax=1, cmap='viridis')
# vols[sub+'_lin_fold1_stderr'] = cortex.Volume(lin_fold1, surfaces[sub], transforms[sub], mask=mask, vmin=0, vmax=1, cmap='viridis')
# vols[sub+'_lin_fold2_stderr'] = cortex.Volume(lin_fold2, surfaces[sub], transforms[sub], mask=mask, vmin=0, vmax=1, cmap='viridis')
# vols[sub+'_lin_fold3_stderr'] = cortex.Volume(lin_fold3, surfaces[sub], transforms[sub], mask=mask, vmin=0, vmax=1, cmap='viridis')

# vols[sub+'_mlp_fold0_stderr'] = cortex.Volume(mlp_fold0, surfaces[sub], transforms[sub], mask=mask, vmin=0, vmax=1, cmap='viridis')
# vols[sub+'_mlp_fold1_stderr'] = cortex.Volume(mlp_fold1, surfaces[sub], transforms[sub], mask=mask, vmin=0, vmax=1, cmap='viridis')
# vols[sub+'_mlp_fold2_stderr'] = cortex.Volume(mlp_fold2, surfaces[sub], transforms[sub], mask=mask, vmin=0, vmax=1, cmap='viridis')
# vols[sub+'_mlp_fold3_stderr'] = cortex.Volume(mlp_fold3, surfaces[sub], transforms[sub], mask=mask, vmin=0, vmax=1, cmap='viridis')

# vols[sub+'_mlp_allvoxels_fold0_stderr'] = cortex.Volume(mlp_allvoxels_fold0, surfaces[sub], transforms[sub], mask=mask, vmin=0, vmax=1, cmap='viridis')
# vols[sub+'_mlp_allvoxels_fold1_stderr'] = cortex.Volume(mlp_allvoxels_fold1, surfaces[sub], transforms[sub], mask=mask, vmin=0, vmax=1, cmap='viridis')
# vols[sub+'_mlp_allvoxels_fold2_stderr'] = cortex.Volume(mlp_allvoxels_fold2, surfaces[sub], transforms[sub], mask=mask, vmin=0, vmax=1, cmap='viridis')
# vols[sub+'_mlp_allvoxels_fold3_stderr'] = cortex.Volume(mlp_allvoxels_fold3, surfaces[sub], transforms[sub], mask=mask, vmin=0, vmax=1, cmap='viridis')


In [ ]:
### END PLOTTING STD DEVIATIONS ####

In [82]:
# vols[sub+'_lin_no_neighborhood'] = cortex.Volume(lin_noneighborhood_loaded.mean(0), surfaces[sub], transforms[sub], mask=mask, vmin=0.5, vmax=0.75, cmap='viridis')
# vols[sub+'_lin_corrs'] = cortex.Volume(lin_corrs_loaded.mean(0), surfaces[sub], transforms[sub], mask=mask, vmin=0, vmax=0.2, cmap='viridis')

# vols[sub+'_mlp_neighborhood'] = cortex.Volume(mlp_neighborhood_loaded.mean(0), surfaces[sub], transforms[sub], mask=mask, vmin=0.5, vmax=0.75, cmap='viridis')
# vols[sub+'_mlp_no_neighborhood'] = cortex.Volume(mlp_noneighborhood_loaded.mean(0), surfaces[sub], transforms[sub], mask=mask, vmin=0.5, vmax=0.75, cmap='viridis')
# vols[sub+'_mlp_corrs'] = cortex.Volume(np.nan_to_num(mlp_corrs_loaded.mean(0)), surfaces[sub], transforms[sub], mask=mask, vmin=0, vmax=0.2, cmap='viridis')

# vols[sub+'_mlpallvox_neighborhood'] = cortex.Volume(mlp_allvoxels_neighborhood_loaded.mean(0), surfaces[sub], transforms[sub], mask=mask, vmin=0.45, vmax=0.55, cmap='viridis')
# vols[sub+'_mlpallvox_no_neighborhood'] = cortex.Volume(mlp_allvoxels_noneighborhood_loaded.mean(0), surfaces[sub], transforms[sub], mask=mask, vmin=0.85, vmax=0.95, cmap='viridis')
# vols[sub+'_mlpallvox_corrs'] = cortex.Volume(mlp_allvoxels_corrs_loaded.mean(0), surfaces[sub], transforms[sub], mask=mask, vmin=-0.1, vmax=0.1, cmap='viridis')

In [93]:
cortex.webshow(vols)

Started server on port 53500


<JS: window.viewer>

Stopping server
